In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
# Plotting 
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Preprocessing
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder

# Metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report,accuracy_score, recall_score, roc_auc_score, precision_score
from sklearn.metrics import roc_curve, auc

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMClassifier 

# Model Tuning 
from bayes_opt import BayesianOptimization

# Feature Importance 
import shap

# Ignore Warnings 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(train.shape , test.shape)

In [ ]:
data = pd.concat([train.drop(['id','target'], axis = 1), test.drop('id', axis = 1)], axis = 0)
data

In [ ]:
data.info()

In [ ]:
corrmat = train.corr()
plt.figure(figsize=(32, 16))
sns.heatmap(corrmat, annot = True, fmt='.2f');
corrmat

In [ ]:
data.isna().sum()

In [ ]:
features = data.columns.tolist()
features

In [ ]:
cat_features = train.select_dtypes(['object']).columns.to_list()
cat_features

In [ ]:
for feature in cat_features:
    le = LabelEncoder()
    le.fit(data[feature])
    data[feature] = le.transform(data[feature])

In [ ]:
data

In [ ]:
# Create test and train set 80-20

# Sepearte features and target from data_csv
X = data.iloc[:300000]
y = train['target']

# train-test stratified split using 80-20 
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2, random_state = 0, shuffle= True,stratify = y)

In [ ]:
# Create initial models
LogReg = LogisticRegression(random_state=0).fit(train_X, train_y)
XGBClass = xgb.XGBClassifier(eval_metric  = "logloss", max_depth=5, learning_rate=0.01, n_estimators=1000, gamma=0, 
                        min_child_weight=1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.005,seed = 0).fit(train_X,train_y)
RFClass = RandomForestClassifier(n_estimators = 1000, max_depth = 50,n_jobs = -1, random_state = 0).fit(train_X,train_y)
LGBMClass = LGBMClassifier(random_state=0).fit(train_X, train_y)

In [ ]:
pred_y = LogReg.predict(valid_X)
print("                    Logistic Regression")
print(classification_report(valid_y,pred_y,digits=3))

pred_y = XGBClass.predict(valid_X)
print("                    XGBoost Classifier")
print(classification_report(valid_y,pred_y,digits=3))

pred_y = RFClass.predict(valid_X)
print("                    Random Forest Classifier")
print(classification_report(valid_y,pred_y,digits=3))

pred_y = LGBMClass.predict(valid_X)
print("                    LightGBM Classifier")
print(classification_report(valid_y,pred_y,digits=3))

In [ ]:
# Create the LightGBM data containers
# Make sure that cat_features are used
train_lgbdata=lgb.Dataset(train_X,label=train_y, categorical_feature = cat_features,free_raw_data=False)
test_lgbdata=lgb.Dataset(valid_X,label=valid_y, categorical_feature = cat_features,free_raw_data=False)

In [ ]:
# https://github.com/fmfn/BayesianOptimization
def search_best_param(X,y,cat_features):
    
    trainXY = lgb.Dataset(data=X, label=y,categorical_feature = cat_features,free_raw_data=False)
    # define the lightGBM cross validation
    def lightGBM_CV(max_depth, num_leaves, n_estimators, learning_rate, subsample, colsample_bytree, 
                lambda_l1, lambda_l2, min_child_weight):
    
        params = {'boosting_type': 'gbdt', 'objective': 'binary', 'metric':'auc', 'verbose': -1,
                  'early_stopping_round':100}
        
        params['max_depth'] = int(round(max_depth))
        params["num_leaves"] = int(round(num_leaves))
        params["n_estimators"] = int(round(n_estimators))
        params['learning_rate'] = learning_rate
        params['subsample'] = subsample
        params['colsample_bytree'] = colsample_bytree
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_child_weight'] = min_child_weight
    
        score = lgb.cv(params, trainXY, nfold=5, seed=1, stratified=True, verbose_eval =False, metrics=['auc'])
        return np.mean(score['auc-mean']) # maximize auc-mean

    # use bayesian optimization to search for the best hyper-parameter combination
    lightGBM_Bo = BayesianOptimization(lightGBM_CV, 
                                       {
                                          'max_depth': (5, 50),
                                          'num_leaves': (20, 100),
                                          'n_estimators': (100, 1000),
                                          'learning_rate': (0.01, 0.3),
                                          'subsample': (0.7, 0.8),
                                          'colsample_bytree' :(0.5, 0.99),
                                          'lambda_l1': (0, 5),
                                          'lambda_l2': (0, 3),
                                          'min_child_weight': (2, 50) 
                                      },
                                       random_state = 1,
                                       verbose = 3
                                      )
    np.random.seed(1)
    
    lightGBM_Bo.maximize(init_points= 5, n_iter=5) # 5 + 5, 10 iterations 
    # n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    # init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    # more iterations more time spent searching 
    
    params_set = lightGBM_Bo.max['params']
    
    # get the params of the maximum target     
    max_target = -np.inf
    for i in lightGBM_Bo.res: # loop thru all the residuals 
        if i['target'] > max_target:
            params_set = i['params']
            max_target = i['target']
    
    params_set.update({'verbose': -1})
    params_set.update({'metric': 'auc'})
    params_set.update({'boosting_type': 'gbdt'})
    params_set.update({'objective': 'binary'})
    
    params_set['max_depth'] = int(round(params_set['max_depth']))
    params_set['num_leaves'] = int(round(params_set['num_leaves']))
    params_set['n_estimators'] = int(round(params_set['n_estimators']))
    params_set['seed'] = 1 #set seed
    
    return params_set

In [ ]:
# Search for best param on Training Set
best_params = search_best_param(train_X,train_y,cat_features)

In [ ]:
# Print best_params
for key, value in best_params.items():
    print(key, ' : ', value)

In [ ]:
# Train lgbm_best using the best params found from Bayesian Optimization
lgbm_best = lgb.train(best_params,
                 train_lgbdata,
                 num_boost_round = 5000,
                 valid_sets = test_lgbdata,
                 early_stopping_rounds = 500,
                 verbose_eval = 25
                 )

In [ ]:
pred_test = lgbm_best.predict(data.iloc[300000:])
pred_test

In [ ]:
sub = pd.DataFrame()
sub['id'] = test['id']
sub['target'] = pred_test
sub.to_csv('submit2.csv',index = False)